# Link marketing data with S&P500 and calculate sigma(mainly)
## link with s&p500
- Return_with_delist.csv is from former work
- sp_500.csv is downloaded from web

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
ret = pd.read_csv('data/Return_with_delist.csv')
sp500 =pd.read_csv('data/sp_500.csv')

In [ ]:
#calculate mdate ,that is : translate the date int he form yyyymm
def mdate(x):
    x = str(x)
    strp = datetime.datetime.strptime(x,'%Y%m%d')
    year = strp.year
    month =strp.month
    mdate = year*100 + month
    return mdate

In [ ]:
#merge sp500 with the return_with_delist 
sp500_mdate = map(mdate,sp500['DATE'])
sp500['mdate'] =sp500_mdate
sp500 = sp500.drop('DATE',axis = 1)
join = pd.merge(ret,sp500,on='mdate',how = 'left')

In [ ]:
#replace the original price with the absolute value of it,for future use
def abs_try(x,y):
    if pd.notnull(x):
        return abs(y) 
prc_abs=map(abs_try,join['RET'],join['PRC'])
join.drop('PRC',axis =1)
join['PRC'] = prc_abs

In [ ]:
#define a function to transfer the return into float type, if it's null , replace it with zero
def transfloat(x):
    try:
        x = float(x)
        return x
    except:
        return 0

In [ ]:
#replace the original return with the float type one
market = join
RET = map(transfloat,market['RET'])
market.drop('RET',axis =1)
market['RET'] = RET

In [ ]:
market.columns = [u'DATE', u'RET', u'SHROUT', u'mdate', u'PRC', u'PERMNO', u'CUSIP',
       u'DLSTDT', u'DLSTCD', u'DLPRC', u'DLRET', u'vwretd', u'vwretx',
       u'ewretd', u'ewretx', u'sprtrn', u'totval', u'totcnt']

## Calculate sigma
- sigma is the zero-centered variance for a period of time
- in this section , we calculated the variance of the last 3 months as the volatitily of the current month

In [ ]:
#calculate the date after a certain lag
def dat_lag(x,lagtime):
    time_after_lag_1 = x - datetime.timedelta(days = lagtime)
    date_lag_1m = time_after_lag_1.year * 100 + time_after_lag_1.month
    return date_lag_1m
def lag(lagtime1,lagtime2,lagtime3,datlist):
    lag1 = [lagtime1] * len(datlist)
    lag2 = [lagtime2] * len(datlist)
    lag3 = [lagtime3] * len(datlist)
    return lag1,lag2,lag3
# calculate the zero-centered sigma for the a certain period of time
def df_sigma(lagtime1,lagtime2,lagtime3,df):
    df['DATE'] = df['DATE'].astype(str)
    dat = map(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d'), df['DATE'])
    lag1,lag2,lag3 = lag(lagtime1,lagtime2,lagtime3,dat)
    lag1mon = map(dat_lag,dat,lag1)
    lag2mon = map(dat_lag,dat,lag2)
    lag3mon = map(dat_lag,dat,lag3)
    df['lag1mon'] = lag1mon
    df['lag2mon'] = lag2mon
    df['lag3mon'] = lag3mon
    df_lag1 = df.loc[:,['PERMNO','RET','lag1mon']]
    df_lag2 = df.loc[:,['PERMNO','RET','lag2mon']]
    df_lag3 = df.loc[:,['PERMNO','RET','lag3mon']]
    df_lag1.columns = [u'PERMNO', u'RET1m', u'mdate']
    df_lag2.columns = [u'PERMNO', u'RET2m', u'mdate']
    df_lag3.columns = [u'PERMNO', u'RET3m', u'mdate']
    df_join1 = pd.merge(df,df_lag1,on = ['PERMNO','mdate'],how = 'inner')
    df_join2 = pd.merge(df_join1,df_lag2,on = ['PERMNO','mdate'],how = 'inner')
    df_join3 = pd.merge(df_join2,df_lag3,on = ['PERMNO','mdate'],how = 'inner')
    sigma = map(lambda x,y,z:np.sqrt((x*x+y*y+z*z)/3),df_join3['RET1m'],df_join3['RET2m'],df_join3['RET3m'])
    df_join3['sigma'] = sigma
    return df_join3

In [ ]:
market = df_sigma(31,62,93,market)#calculate the sigma for the last 3 months
market= market.drop(['RET1m','RET2m','RET3m',u'lag1mon',u'lag2mon',  u'lag3mon'],axis=1)

In [ ]:
##set the null value in column(sigma) to 0##
alist = market['sigma'].isnull()
alist = np.array(alist)
sigma = np.array(market['sigma'])
sigma[alist] = 0
market.drop('sigma',axis = 1)
market['sigma'] = sigma
#

In [ ]:
market.to_csv('market_raw.csv')#write to market_raw.csv

# annulized monthly data and calculate several features
## anualize monthly data
- for for 'ret' and 'vwretd', we used factorial(ret) and factorial(vwretd)
- for the rest , we use average(rest)

In [ ]:
raw = pd.read_csv('data/market_raw.csv')

In [ ]:
#select a certain company‘s dataframe
def get_df(ret,permno):
    df = ret[ret['PERMNO'] == permno]
    return df
#identify the year
def indexer(df):
    c = df.groupby('fyear').apply(list)
    d = c.keys()
    return d
#for return and vwretd, we need to calculate the floor of the 12 months, like Rm1*Rm2*...Rm12
def ret(df,column):
    grouped = df.groupby('fyear')[column].apply(np.array)
    value = grouped.values
    a = [i.resize((1,12),refcheck = False) for i in value]
    aa = [np.squeeze(i) for i in value]
    aaplu1 = [i+1 for i in aa]
    ret = [np.prod(i) for i in aaplu1]
    return ret
#calculate the average sigma for a year
def sigma(df): 
    grouped = df.groupby('fyear')['RET'].apply(np.array)
    value = grouped.values
    a = [i.resize((1,12),refcheck = False) for i in value]
    aa = [np.squeeze(i) for i in value]
    dot = [np.dot(i,i)/12 for i in aa]    
    return dot
#average a year's value
def average(df,column):
    grouped = df.groupby('fyear')[column].apply(np.array)
    value = grouped.values
    averaged = [np.sum(i)/12 for i in value]
    return averaged

In [ ]:
def dlst(df):#to detect if there is a delist in a year
    grouped = df.groupby('fyear')['delist'].apply(np.array)
    value = grouped.values
    de = map(lambda x:np.sum(x),value)
    return de

In [ ]:
def merge_permno(dataframe,permno):#calculate the dataframe for a certain company
    df = get_df(dataframe,permno)
    INDEXER = indexer(df)
    prc = average(df,'prc2')
    totval = average(df,'totval')
    MKET = average(df,'mket')
    SIGMA = sigma(df)
    RET = ret(df,'RET')
    vwretd = ret(df,'vwretd')
    DELIST = dlst(df)
    PERMNO = [permno] * len(INDEXER)
    dat = pd.DataFrame({'fyear':INDEXER,
                        'PERMNO':PERMNO,
                        'PRC':prc,
                        'totval':totval,
                        'sigma':SIGMA,
                        'RET':RET,
                        'vwretd':vwretd,
                        'mket': MKET,
                        'DELISTorNOT':DELIST})
    return dat

In [ ]:
def merge_final(dataframe):# calculation for all the companies
    origin = merge_permno(dataframe,10000)
    for i in range(0,len(permno_list)):
        tmp = merge_permno(market,permno_list[i])
        origin =origin.append(tmp)
    return origin

## calculate several features
- prc2,mkt,exrcamp

In [ ]:
#calculate the feature prc2
def price2(price):
    if price>15:
        prc2 = 15
    else:
        prc2 = price
    prc2 = np.log(prc2)
    return prc2
def prc2_map(dataset):
    prc2_result= map(price2,dataset['PRC'])
    return prc2_result
#calculate the feature mkt
def mkt(prc,shrout):
    equity = prc * shrout
    mket = equity/1000
    return mket
def mkt_map(dataset):
    mkt_result = map(mkt,dataset['PRC'],dataset['SHROUT'])
    return mkt_result
prc2 = prc2_map(raw)
mket = mkt_map(raw)

In [ ]:
#calculate the feature exrcamp = LOG(1+ret)-LOG(1+vwretd)
def func(ret,vwretd):
    tmp = np.log(1+ret) - np.log(1+vwretd)
    return tmp
def exrcamp(dataset):
    result = map(func,dataset['RET'],dataset['vwretd'])
    return result
exrca = exrcamp(raw)

In [ ]:
raw['prc2']=prc2
raw['mket']=mket
raw['excamp']=exrca
market = raw[raw['mket'].notnull()]
market =market[market['excamp'].notnull()]
market = market.fillna(0)
delist = map(deornot,market['DLSTCD'])
market.columns=[u'Unnamed: 0', u'DATE', u'RET', u'SHROUT', u'mdate', u'PRC', u'PERMNO',
       u'cusip', u'DLSTDT', u'DLSTCD', u'DLPRC', u'DLRET', u'vwretd',
       u'vwretx', u'ewretd', u'ewretx', u'sprtrn', u'totval', u'totcnt',
       u'prc2', u'mket', u'excamp', u'delist']
date = map(lambda x:pd.to_datetime(x),market['DATE'])
year = map(lambda x:x.year, date)
market['fyear'] = year
market['delist'] = delist

In [ ]:
permno_list = market['PERMNO'].unique()
final = merge_final(market)#get the final anualized dataframe
final.to_csv('anualized_marketing_data.csv')#write to anulized_market_data.csv

# Adjust the marketing data
- because in former work , the delist label of a delisted company is 1 for every fiscal year, in this section ,for a certain delisted company, we will set the delist label for the last year to 1 and rest to 0. 

In [ ]:
mkt = pd.read_csv('data/anualized_marketing_data.csv')
mkt = mkt.drop('Unnamed: 0',axis=1)
mkt.columns=[u'DELISTorNOT', u'PERMNO', u'PRC', u'RET', u'mket', u'sigma', u'totval',
       u'vwretd', u'fyear']

In [ ]:
#because from the former work ,value in DELISTorNOT is range from 0-12, here we need to 
#create a new feature D to denote whether it is delisted or not
def D_mkt(lis):# transform from '0' and '1-12' as '0' or '1'
    if lis == 0:
        return 0
    else:
        return 1
d_mkt = map(D_mkt,mkt['DELISTorNOT'])
mkt['D'] = d_mkt
mkt_isin = mkt[mkt['D'] == 1]
mkt_notin= mkt[mkt['D'] != 1]

In [ ]:
#check if the year is the latest one,if it is ,return 1, if it is not ,return 0
def year_zero(newest,year): 
    if year < newest:
        return 0
    else:
        return 1
#before, for a delisted company , D=1 for all the fyear, here , we only set the latest one --
# i.e. the year it is delisted -- as 1, set rest as 0
def firm_zero(onefirm):
    maxn = max(onefirm['fyear'])
    newest = [maxn]*len(onefirm)
    ddd = map(year_zero,newest,onefirm['fyear'])
    onefirm['D'] = ddd
    return onefirm   
#select a specific firm by permno
def get_firm(dataset,permno):
    firmdf = dataset[dataset['PERMNO'] == permno]
    return firmdf
def whole_zero(isin):# only let the last D be 1, others 0
    origin = isin['PERMNO'].unique()
    first = origin[0]
    permno_list = origin[1:]
    firstfirm = get_firm(isin,first)
    firstdf = firm_zero(firstfirm)
    for i in range(0,len(permno_list)):
        firm = get_firm(isin,permno_list[i])
        firmdf = firm_zero(firm)
        firstdf=firstdf.append(firmdf)
        
    return firstdf
mkt_isin_1= whole_zero(mkt_isin)

In [ ]:
mkt_final = mkt_isin_1.append(mkt_notin)
mkt_final.to_csv('mkt_final.csv')